In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
}

defined trait KnightsKnaves

# Puzzle 1 - Ejemplo.
Is it possible for any inhabitant of this island to claim that he is a knave?


No; no inhabitant can claim to be a knave because no knight would lie and say he is a knave and no knave would truthfully admit to being a knave.

In [5]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): ¬[x.Says[x.Knave]] = 
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [6]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): x.Says[x.Knave] => Nothing =
    // 1. Says(x, Knave(x))                                       ; hypothesis
    ((_1: x.Says[x.Knave]) => 
         (premises.P1(x) match {
             // 2. Knight(x)                                      ; hypothesis
             case Left(_2: x.Knight) => 
                 //   3. Knight(x) → Says(x, Knave(x)) → Knave(x) ; P2[Knave(x),x]
                 val _3: x.Knight => x.Says[x.Knave] => x.Knave = premises.P2[x.Knave](x)
                 //   4. Says(x, Knave(x)) → Knave(x)             ; ⟶E(3,2)
                 val _4: x.Says[x.Knave] => x.Knave = _3(_2)
                 //   5. Knave(x)                                 ; ⟶E(4,1)
                 //   5. ¬ Knight(x)                              ; ≝ Knave
                 //   5. Knight(x) → ⊥                            ; ≝ ¬ 
                 val _5: x.Knight => ⊥ = _4(_1)
                 //   6. ⊥                                        ; ⟶E(5,2)
                 _5(_2) : ⊥

             //  _7. Knave(x)                                     ; hypothesis
             case Right(_7: x.Knave) => 
                 //   8. Knave(x) → Says(x, Knave(x)) → ¬ Knave(x) ; P3[Knave(x),x]
                 val _8: x.Knave => x.Says[x.Knave] => ¬[x.Knave] = premises.P3[x.Knave](x)
                 //   9. Says(x, Knave(x)) → ¬ Knave(x)            ; ⟶E(8,7)
                 val _9: x.Says[x.Knave] => ¬[x.Knave] = _8(_7)
                 //   10. ¬ Knave(x)                               ; ⟶E(9,1)
                 //   10. Knave(x) → ⊥                             ; ≝ ¬ 
                 val _10: x.Knave => ⊥ = _9(_1)
                 //  11. ⊥                                         ; ⟶E(10,7)
                 _10(_7) : ⊥

        // _12. ⊥                                                  ; ∨E(P1, 2-6, 7-11)
        }) : False

    // _13. Says(x, Knave(x)) ⟶ ⊥                                  ; ⟶I(1-12)
    // _13. ¬ Says(x, Knave(x))                                    ; ≝ ¬ 
    ) : ¬[x.Says[x.Knave]]

defined function puzzle1

In [7]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = P.P1(x)
    def knightsAreTruthTellers[P](x: Inhabitant) = P.P2[P](x)
    def knavesAreLiers[P](x: Inhabitant) = P.P3[P](x)
    
    def noKnightLies[P](x: Inhabitant): x.Knight => x.Says[Not[P]] => P => False = 
        xIsKnight => xSaysNotP => p => 
            P.P2[Not[P]](x)(xIsKnight)(xSaysNotP)(p)
    
    def noKnaveTellsTruth[P](x: Inhabitant): x.Knave => x.Says[P] => P => False = 
        xIsKnave => xSaysP => p => 
            P.P3[P](x)(xIsKnave)(xSaysP)(p)
}

defined class SomeSugar

In [8]:
// { P1, P2, P3 } ⊢ ∀ x. Inhabitant(x) ⟶ ¬ Says(x, Knave(x))

def puzzle1(premises: KnightsKnaves)(x: Inhabitant): Not[x.Says[x.Knave]] =
    xSaysIsKnave =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => 
                // no knight would lie
                premises.noKnightLies(x)(xIsKnight)(xSaysIsKnave)(xIsKnight),
            xIsKnave => 
                // no knave would tell the truth
                premises.noKnaveTellsTruth(x)(xIsKnave)(xSaysIsKnave)(xIsKnave)
        )

defined function puzzle1

# Puzzles Introducción

Primero se consideran 5 preguntas que servirán como introduccion a la logica 
knight-knave para aquellos que no estén familiarizados con ella y como un
breve curso recordatorio para los que lo están.

#### 3. Suppose an inhabitant A says about himself and his brother B: "At least one of us is a knave." What type is A and what type is B?

A says that, of A and B, at least one is a knave. If A were a knave, then it would be true that at least one of A and B is a knave and we would have a knave making a true statement, which is not possible. Therefore A must be a
knight. Since he is a knight, his statement is true, hence at least one really is a knave. It is then B who must be the knave.
So A is a knight and B is a knave.

In [9]:
def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = ???
    /*
    O X es Knave o Y es Knave. En la premisa solo dice que al menos uno es Knave. No dice al menos uno es Knave y el otro
    es Knight o viceversa.
    */
    //x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = ???
    /* Al menos uno de ellos es un Knave (uno u otro, o ambos):
    1.- X es Knave
    2.- Y es Knave
    3.- X e Y son Knave
    
    "Uno u otro" -> Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]
    "O ambos" -> Del resultado del "Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]" sacaremos una de las partes del 
    ultimo Either junto a "[Either[...],(x.Knave,y.Knave)]]"
    */



defined function puzzle3

In [10]:
def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = ???

defined function puzzle3

In [11]:
// VERSION 1

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = 
    ({xSay: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = premises.P2[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]) => 
                        (x_3L_left match{
                            case Left(x_3L_left_L: (x.Knave,y.Knight)) => 
                                val xcontradiccion: x.Knight => Nothing = x_3L_left_L._1
                                (xKnight, xcontradiccion(xKnight)) : (x.Knight,y.Knave)
                            case Right(x_3L_left_R: (x.Knight,y.Knave)) =>
                                x_3L_left_R : (x.Knight,y.Knave)
                        }) : (x.Knight,y.Knave)
                    case Right(x_3L_right: (x.Knave,y.Knave)) => 
                        (xKnight, x_3L_right._2) : (x.Knight,y.Knave)
                }) : (x.Knight,y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = premises.P3[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = x_1R(xKnave)
                //val x_3R: ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = x_2R(xSay)
                val x_3R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
            
                // Mi x_2R(xSay) es mi Not[P] del P3[P]
                // Si tengo P y Not[P] entonces tengo una contradiccion. Eso es lo que tengo que encontrar.
                /*
                
                Teniendo el "Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => Nothing"
                Si aplico P1 y P2 a y, puedo sacar "(x.Knave,y.Knight)" para y = Knight y puedo sacar 
                "(x.Knave,y.Knave)" para y = Knave. Osea seria sacar P, que P puede ser:
                (x.Knave,y.Knight) - Left, Left
                (x.Knight,y.Knave) - Left, Right
                (x.Knave,y.Knave) - Right
                
                Y ya con esto, si tengo P (cualquiera de las 3) y tengo Not[P] que es "x_2R(xSay)" entonces tengo una
                contradiccion.
                
                */
                
                {x_4R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => 
                        (x_4R match{
                            case Left(x_4R_left) => 
                                (x_4R_left match{
                                    // ¿Es necesaria la rama Left?
                                    /*
                                    case Left(x_4R_left_left: (x.Knave,y.Knight)) =>
                                        
                                        ??? : Nothing*/
                                    case Right(x_4R_left_right) =>
                                        val x_4R_left_right_contradiccion: x.Knight => Nothing = xKnave
                                        x_4R_left_right_contradiccion(x_4R_left_right._1)
                                }) : Nothing
                            // ¿Es necesaria la rama Right? ¿Como saco una contradiccion?
                            /*
                            case Right(x_4R_right: (x.Knave,y.Knave)) =>
                                
                                ??? : Nothing*/
                }) : Nothing}
                
                
                ??? : (x.Knight,y.Knave)
        })
        
        
    ??? : (x.Knight,y.Knave)})

defined function puzzle3

In [12]:
// VERSION 1

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = 
    ({xSay: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
        (premises.P1(x) match{
            case Left(xKnight: x.Knight) => 
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                val x_1L: x.Knight => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = premises.P2[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                val x_2L: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_1L(xKnight)
                val x_3L: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_2L(xSay)
                
                (x_3L match{
                    case Left(x_3L_left: Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]) => 
                        (x_3L_left match{
                            case Left(x_3L_left_L: (x.Knave,y.Knight)) => 
                                val xcontradiccion: x.Knight => Nothing = x_3L_left_L._1
                                (xKnight, xcontradiccion(xKnight)) : (x.Knight,y.Knave)
                            case Right(x_3L_left_R: (x.Knight,y.Knave)) =>
                                x_3L_left_R : (x.Knight,y.Knave)
                        }) : (x.Knight,y.Knave)
                    case Right(x_3L_right: (x.Knave,y.Knave)) => 
                        (xKnight, x_3L_right._2) : (x.Knight,y.Knave)
                }) : (x.Knight,y.Knave)
            
            case Right(xKnave: x.Knave) => 
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                val x_1R: x.Knave => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = premises.P3[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                val x_2R: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = x_1R(xKnave)
                //val x_3R: ¬[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = x_2R(xSay)
                val x_3R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => Nothing = x_2R(xSay)
                
                x_4R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => 
                        (x_4R match{
                            case Left(_) => 
                                x_3R(x_4R) : Nothing
                            case Right(_) =>
                                x_3R(x_4R) : Nothing
                        }) : Nothing
                // ¿Porqué teniendo ya algo de tipo (x.Knight,y.Knave) no puedo quitar las ultimas ??? ?
                // ¿Si no obtengo solucion en esta rama es porque la respuesta correcta es solo si x es Knight ?
                (x_2R(xSay)(x_4R), x_2R(xSay)(x_4R)): (x.Knight,y.Knave)
                //(x_3R(x_4R): x.Knight, x_3R(x_4R): y.Knave) : (x.Knight,y.Knave) 
            
        })
    ??? : (x.Knight,y.Knave)})

defined function puzzle3

In [13]:
// SIMPLIFICADO VERSION 1
def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = 
    (premises.P1(x) match {
        case Left(xKnight) => 
            (xsaysL) => 
                    val x_1L: x.Knight => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = premises.P2[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                    val x_2L: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_1L(xKnight)
                    val x_3L: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] = x_2L(xsaysL)
                    (x_3L match{
                        case Left(x_3L_left) => 
                            (x_3L_left match{
                                case Left(x_3L_left_left) => 
                                    val xcontradiccion: x.Knight => Nothing = x_3L_left_left._1
                                    (xKnight, xcontradiccion(xKnight))
                                case Right(x_3L_left_right) => 
                                    x_3L_left_right
                            }) : (x.Knight,y.Knave)
                        case Right(x_3L_right) => 
                            (xKnight, x_3L_right._2)
                    }) : (x.Knight,y.Knave)
        
        case Right(xKnave: x.Knave) => 
            (xsaysR) => 
                    val x_1R: x.Knave => 
                            x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Not[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = premises.P3[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]](x)
                    val x_2R: x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => 
                                Not[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] = x_1R(xKnave)
                    val x_3R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => Nothing = x_2R(xsaysR)
                    
                    {x_4R: Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)] => 
                        (x_4R match{
                            case Left(x_4R_left) => 
                                (x_4R_left match{
                                    // ¿Es necesaria la rama Left?
                                    /*
                                    case Left(x_4R_left_left: (x.Knave,y.Knight)) =>
                                        
                                        ??? : Nothing*/
                                    case Right(x_4R_left_right) =>
                                        val x_4R_left_right_contradiccion: x.Knight => Nothing = xKnave
                                        x_4R_left_right_contradiccion(x_4R_left_right._1)
                                }) : Nothing
                            // ¿Es necesaria la rama Right? ¿Como saco una contradiccion?
                            /*
                            case Right(x_4R_right: (x.Knave,y.Knave)) =>
                                
                                ??? : Nothing*/
                        }) : Nothing}
            ??? : (x.Knight,y.Knave)
        
    }): (x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave))

defined function puzzle3

In [14]:
//VERSION 2

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = ???
    /*
    CORRECCION JUANMA:
    
    A dice "At least one of us is a knave".
    Case 1) Suponemos que A es Knight:
    
    Como A es Knight, dice la verdad, por tanto "At least one of us is a knave" se cumple. Asi que si A es Knight, quiere
    decir que B sera Knave.
    
    Case 2) Suponemos que A es Knave:
    
    Como A es Knave, quiere decir que siempre miente. Asi que, "At least one of us is a knave" lo tomamos como que es mentira.
    Lo que significa que A y B son ambos knights, lo cual es una contradiccion porque A es knave. Asi que no es posible que
    A sea un Knave.
    
    Conclusion, "At least one of us is a knave" se cumple si A es knight y B es knave.    
    _________________________
    
    O X es Knave o Y es Knave. En la premisa solo dice que al menos uno es Knave. No dice al menos uno es Knave y el otro
    es Knight o viceversa.
    */
    //x.Says[Either[Either[(x.Knave,y.Knight),(x.Knight,y.Knave)],(x.Knave,y.Knave)]] => (x.Knight,y.Knave) = ???
    /* Al menos uno de ellos es un Knave (uno u otro, o ambos):
    1.- X es Knave
    2.- Y es Knave
    3.- X e Y son Knave
    
    "Uno u otro" -> Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]
    "O ambos" -> Del resultado del "Either[(x.Knave,y.Knight),(x.Knight,y.Knave)]" sacaremos una de las partes del 
    ultimo Either junto a "[Either[...],(x.Knave,y.Knave)]]"
    */



defined function puzzle3

In [15]:
//VERSION 2

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays : x.Says[Either[x.Knave, y.Knave]] => 
            ??? : (x.Knight,y.Knave)})

defined function puzzle3

In [16]:
//VERSION 2

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays : x.Says[Either[x.Knave, y.Knave]] => 
            (premises.P1(x) match{
                case Left(xKnight: x.Knight) =>
                // def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    (xKnight: x.Knight, (x_3L match{
                        case Left(xKnave: x.Knave) => 
                            val aux_contradiccion: x.Knight => Nothing = xKnave
                            aux_contradiccion(xKnight) : y.Knave
                        case Right(yKnave: y.Knave) => 
                            yKnave : y.Knave
                    }): y.Knave) : (x.Knight,y.Knave)
                                
                case Right(xKnave: x.Knave) =>
                //def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    //val x_3R: Not[Either[x.Knave, y.Knave]] = x_2R(xSays)
                
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                    //Si a la funcion x_3R le pasas o un x.Knave o un y.Knave devuelve Nothing 
                    //Aqui tengo P que es "Either[x.Knave, y.Knave]" y tengo Not[P] que es "x_2R(xSays)",
                    //por tanto he demonstrado la contradiccion
                    x_3R(Left(xKnave)): Nothing
               
                (x_3R(Left(xKnave)), x_3R(Left(xKnave))) : (x.Knight,y.Knave)
                
            }) : (x.Knight,y.Knave)})

defined function puzzle3

In [17]:
//VERSION 2 - SIMPLIFICADA

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    (xKnight, (x_3L match{
                        case Left(xKnave: x.Knave) => 
                            val aux_contradiccion: x.Knight => Nothing = xKnave
                            aux_contradiccion(xKnight) : y.Knave
                        case Right(yKnave: y.Knave) => 
                            yKnave : y.Knave
                    }))
                                
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                                   
                    (x_3R(Left(xKnave)), x_3R(Left(xKnave)))
            }) 
        })

defined function puzzle3

In [22]:
//VERSION 2 - SIMPLIFICADA 2.0

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                                   
                    //(x_3R(Left(xKnave)), x_3R(Left(xKnave)))
                    x_3R(Left(xKnave)): (x.Knight,y.Knave)
            }) 
        })

defined function puzzle3

In [22]:
//Sugar version

/*def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = premises.P3[Either[x.Knave, y.Knave]](x)
                    val x_2R: x.Says[Either[x.Knave, y.Knave]] => Not[Either[x.Knave, y.Knave]] = x_1R(xKnave)
                    val x_3R: Either[x.Knave, y.Knave] => Nothing = x_2R(xSays)
                                   
                    (x_3R(Left(xKnave)), x_3R(Left(xKnave)))
            }) 
        })
*/

def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        xSay_xKnaveOryKnave =>
        //({xSays =>
            premises.eitherKnightOrKnave(x).fold(
            //(premises.P1(x) match{
                xIsKnight =>
                //case Left(xKnight) =>
                    val aux1 = premises.noKnightLies(x)(xIsKnight)(xSay_xKnaveOryKnave: x.Says[Either[x.Knave, y.Knave]]): Either[x.Knave, y.Knave]
                    aux1 match{
                        xKnave => xKnave(xIsKnight)
                        yKnave => (xIsKnight, yKnave)
                    }              
                ,
                xIsKnave =>
                //case Right(xKnave) =>
                    val aux2 = premises.no KnaveTellsTruth(x)(xIsKnave)(xSay_xKnaveOryKnave: x.Says[Either[x.Knave, y.Knave]]): Not[x.Says[Either[x.Knave, y.Knave]]]
                    aux2(Left(xIsKnave))            
            
            )

(console):35:21 expected ")"
//Sugar version

/*def puzzle3(premises: KnightsKnaves)(x: Inhabitant)(y: Inhabitant): 
    x.Says[Either[x.Knave, y.Knave]] => (x.Knight,y.Knave) = 
        ({xSays => 
            (premises.P1(x) match{
                case Left(xKnight) =>
                    val x_1L: x.Knight =>
                                x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = premises.P2[Either[x.Knave, y.Knave]](x)
                    val x_2L: x.Says[Either[x.Knave, y.Knave]] => Either[x.Knave, y.Knave] = x_1L(xKnight)
                    val x_3L: Either[x.Knave, y.Knave] = x_2L(xSays)
                    
                    x_3L match{
                        case Left(xKnave: x.Knave) => xKnave(xKnight) : (x.Knight,y.Knave)
                        case Right(yKnave: y.Knave) => (xKnight, yKnave) : (x.Knight,y.Knave)
                    }
                case Right(xKnave) =>
                    val x_1R: x.Knave => x.Says[Either[x.Knave, y.Knave]] =>

: 